# GA Capstone
## Causal Language Transformer Modeling with GPT2

Creating a model to generate text

A lot of the below is adapted from the gpt2 tutorial at https://huggingface.co/docs/transformers/v4.22.2/en/tasks/language_modeling

### Imports and Preliminaries

In [1]:
# data formatting for model
from datasets import Dataset, DatasetDict

# train/test split
from sklearn.model_selection import train_test_split

# tokenizer
from transformers import AutoTokenizer

# lm collator
from transformers import DataCollatorForLanguageModeling

# model and support
from transformers import TFAutoModelForCausalLM, create_optimizer, AdamWeightDecay

# other utilities
from itertools import chain
import os
import random
import re
import json

In [2]:
# set the model we are using
# distilgpt2: smallest - 84M features
# gpt2: normal - 117M features
# gpt2-medium: big - 355M features
# and 2 more tiers with even more features up to 1.5B I believe

# using distilgpt2 for all tests, will try gpt2 for production
model_type = 'distilgpt2' 

In [3]:
# directories and other constants, from config.json

with open('config.json', 'r') as jsf:
    config_vars = json.load(jsf)

MODEL_NAME = config_vars['DEFAULT_MODEL_NAME']
N_EPOCHS = config_vars['DEFAULT_N_EPOCHS']
DIR_MODEL = config_vars['MODEL_DIR']
DIR_DATA = config_vars['DATA_DIR']

In [4]:
# regexes
RE_SENTENCE = re.compile(r'\w.*?[.?!:;]', re.S)
RE_WHITESPACE = re.compile(r'\s+')
RE_BLANKLINE = re.compile(r'\n\n')

### Load and Format Data

In [5]:
# load data
paths = [os.path.join(DIR_DATA, fn) for fn in config_vars["DATA_SHAKESPEARE"]]

text = list()

for path in paths:
    with open(path, 'r') as f:
        text.append([line.strip() for line in f.readlines() if line.strip()])

text = ' '.join(chain(*text))

In [6]:
# Split into...

# SENTENCES
sentences = RE_SENTENCE.findall(text)
sentences = [RE_WHITESPACE.sub(' ', sentence) for sentence in sentences]

print(f'Number of Sentences: {len(sentences)}')

Number of Sentences: 76578


In [7]:
# reduce sample size if set in config
nsamps = config_vars['N_SAMPLES'] or len(sentences)
sentences = sentences[:nsamps]
print(f'Using {nsamps} of {len(sentences)} samples')

Using 76578 of 76578 samples


In [8]:
# split train and test
lines_train, lines_test = train_test_split(sentences, test_size=0.05, shuffle=False)
len(lines_train), len(lines_test)

(72749, 3829)

### Cleaning and Data Preparation

In [9]:
# create Dataset and DatasetDict instances - I think this is needed for model
train_dataset = Dataset.from_dict({'text': lines_train})
test_dataset = Dataset.from_dict({'text': lines_test})
datasets = DatasetDict({'train': train_dataset, 'test': test_dataset})
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 72749
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3829
    })
})

In [10]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_type)

In [11]:
# Preprocessing function for tokenizer to use with map() method of datasetdict
def token_preproc(data):
    return tokenizer(data['text'])

In [12]:
# tokenize data
tokened_data = datasets.map(token_preproc, batched=True, num_proc=4, remove_columns=['text'])
tokened_data

#0:   0%|          | 0/19 [00:00<?, ?ba/s]

#1:   0%|          | 0/19 [00:00<?, ?ba/s]

#2:   0%|          | 0/19 [00:00<?, ?ba/s]

#3:   0%|          | 0/19 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 72749
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3829
    })
})

In [13]:
for _ in range(4):
    n = random.randint(0, len(tokened_data['train']))
    print(n, tokenizer.convert_ids_to_tokens(tokened_data['train'][n]['input_ids']), lines_train[n])

13380 ['Stand', ',', 'ĠOr', 'Ġwe', 'Ġare', 'ĠRomans', 'Ġand', 'Ġwill', 'Ġgive', 'Ġyou', 'Ġthat', 'ĠLike', 'Ġbeasts', 'Ġwhich', 'Ġyou', 'Ġshun', 'Ġbeast', 'ly', ',', 'Ġand', 'Ġmay', 'Ġsave', 'ĠBut', 'Ġto', 'Ġlook', 'Ġback', 'Ġin', 'Ġfrown', '.'] Stand, Or we are Romans and will give you that Like beasts which you shun beastly, and may save But to look back in frown.
37814 ['This', 'Ġis', 'Ġab', 'h', 'omin', 'able', 'âĢĶ', 'which', 'Ġhe', 'Ġwould', 'Ġcall', 'Ġab', 'omin', 'able', '.'] This is abhominable—which he would call abominable.
22808 ['But', 'Ġlong', 'ĠI', 'Ġwill', 'Ġnot', 'Ġbe', 'ĠJack', '-', 'out', '-', 'of', '-', 'office', '.'] But long I will not be Jack-out-of-office.
48106 ['They', 'Ġnever', 'Ġmeet', 'Ġbut', 'Ġthere', 'âĢ', 'Ļ', 's', 'Ġa', 'Ġskirm', 'ish', 'Ġof', 'Ġwit', 'Ġbetween', 'Ġthem', '.'] They never meet but there’s a skirmish of wit between them.


In [14]:
# pad encodings and prep for modeling
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors='tf')
collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='tf')

### Modeling

In [15]:
# instantiate model
model_path = os.path.join(DIR_MODEL, f'{model_type}.{MODEL_NAME}.{str(N_EPOCHS)}')

if not os.path.exists(model_path):
    model = TFAutoModelForCausalLM.from_pretrained(model_type, pad_token_id = tokenizer.eos_token_id)
else:
    model = TFAutoModelForCausalLM.from_pretrained(model_path)
    
#model.resize_token_embeddings(len(tokenizer))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [16]:
# convert data to special format for tf model
tokenizer.pad_token = tokenizer.eos_token
tf_train_set = model.prepare_tf_dataset(tokened_data['train'], shuffle=True, batch_size=32, collate_fn=collator)
tf_test_set = model.prepare_tf_dataset(tokened_data['test'], shuffle=False, batch_size=32, collate_fn=collator)
tf_train_set

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(32, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(32, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(32, None), dtype=tf.int64, name=None))>

In [17]:
# compile model
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [18]:
# fit model (if pretrained does not exist)
if not os.path.exists(model_path):
    model.fit(tf_train_set, validation_data=tf_test_set, epochs=N_EPOCHS)

Epoch 1/4
 602/2273 [======>.......................] - ETA: 3:56:02 - loss: 4.9713

KeyboardInterrupt: 

In [ ]:
# save model
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.save_pretrained(model_path)

### Test Model

In [ ]:
# function to get predicted text
def test(text, max_new=50, temp=1, top_k=50, rep_penalty=1.5, len_penalty=0.75, n_seq=1):
    tokened = tokenizer(text, return_tensors='tf')
    output = model.generate(**tokened,
                            do_sample=True,
                            max_new_tokens=max_new, 
                            temperature=temp, 
                            top_k=top_k, 
                            repetition_penalty=rep_penalty,
                            length_penalty=len_penalty,
                            num_return_sequences=n_seq)
    return tokenizer.decode(output[0], skip_special_tokens=False)

In [ ]:
test_lines = [sentences[random.randint(0,len(sentences) - 1)] for _ in range(10)]
fragment_ratio = 0.4

for line in test_lines:
    words = line.split()
    stop_ix = int(len(words) * fragment_ratio) or 1
    fragment = ' '.join(words[0:stop_ix])
    print(f'Original: {fragment}')
    output = test(fragment,
                  temp=0.5,
                  max_new=100,
                  top_k=200,
                  rep_penalty=1.5,
                  len_penalty=0.75,
                  n_seq=1)
    print(f'Output: {output}\n')

### Conclusion

Text is generated that at many times sounds reasonably Shakespearian. Soon to be combined with classification model to test this out.